In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = 4.5 * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = (10/3) * TR_C * TR_R
TR_R = tau / ((10/3)*TR_C)

# Logic Gates
g_inv, p_inv = 1, 2.29

g_nand2, g_nand3, g_nand6 = 1.3 , 1.6, 2.5
p_nand2, p_nand3, p_nand6 = 2.29*2, 2.29*3, 2.29*6

g_nor2, g_nor3 = 1.7, 2.4
p_nor2, p_nor3 = 2.29*2, 2.29*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

# NMOS width of repeater derived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/((10/3)*metal_r*TR_C) ) ** (1/2)

C_out = (WL_C + cell_C)/N + (10/3) * W * TR_C
C_in = 5 * 1e-15 * 32

H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = [1,2,4,8,16,32,64,128], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,1,2,4,8,16,32,64,128
NAND6-INV,3.637919,3.14948,2.681211,2.264067,1.957150,1.824577,1.869626,2.030541
NAND3-NOR2,3.698759,3.21032,2.742050,2.324906,2.017989,1.885416,1.930465,2.091381


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.841730e-10,1.519952e-10,1.298605e-10,1.153659e-10,1.070815e-10,1.040133e-10,1.050244e-10,1.089052e-10
NAND3-NOR2 Stage2,1.683887e-10,1.348250e-10,1.117368e-10,9.661798e-11,8.797671e-11,8.477636e-11,8.583103e-11,8.987901e-11


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,1,2,4,8,16,32,64,128
NAND6-INV Stage4,1.562538e-10,1.463637e-10,1.383314e-10,1.321951e-10,1.282143e-10,1.266215e-10,1.271546e-10,1.291281e-10
NAND3-NOR2 Stage4,1.369971e-10,1.268963e-10,1.186928e-10,1.124258e-10,1.083601e-10,1.067334e-10,1.072778e-10,1.092934e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,1,2,4,8,16,32,64,128
NAND6-INV Stage6,1.768164e-10,1.700439e-10,1.642307e-10,1.595561e-10,1.563933e-10,1.550950e-10,1.555317e-10,1.571294e-10
NAND3-NOR2 Stage6,1.562064e-10,1.494339e-10,1.436207e-10,1.389461e-10,1.357833e-10,1.344850e-10,1.349217e-10,1.365194e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND6-INV'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,1,2,4,8,16,32,64,128
NAND6-INV Stage8,2.015863e-10,1.960983e-10,1.912556e-10,1.872605e-10,1.845001e-10,1.833523e-10,1.837394e-10,1.851468e-10
NAND3-NOR2 Stage8,1.816929e-10,1.761468e-10,1.712528e-10,1.672153e-10,1.644256e-10,1.632658e-10,1.636569e-10,1.650793e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,1.841730e-10,1.519952e-10,1.298605e-10,1.153659e-10,1.070815e-10,1.040133e-10,1.050244e-10,1.089052e-10
NAND3-NOR2 Stage2,1.683887e-10,1.348250e-10,1.117368e-10,9.661798e-11,8.797671e-11,8.477636e-11,8.583103e-11,8.987901e-11
NAND6-INV Stage4,1.562538e-10,1.463637e-10,1.383314e-10,1.321951e-10,1.282143e-10,1.266215e-10,1.271546e-10,1.291281e-10
NAND3-NOR2 Stage4,1.369971e-10,1.268963e-10,1.186928e-10,1.124258e-10,1.083601e-10,1.067334e-10,1.072778e-10,1.092934e-10
NAND6-INV Stage6,1.768164e-10,1.700439e-10,1.642307e-10,1.595561e-10,1.563933e-10,1.550950e-10,1.555317e-10,1.571294e-10
NAND3-NOR2 Stage6,1.562064e-10,1.494339e-10,1.436207e-10,1.389461e-10,1.357833e-10,1.344850e-10,1.349217e-10,1.365194e-10
NAND6-INV Stage8,2.015863e-10,1.960983e-10,1.912556e-10,1.872605e-10,1.845001e-10,1.833523e-10,1.837394e-10,1.851468e-10
NAND3-NOR2 Stage8,1.816929e-10,1.761468e-10,1.712528e-10,1.672153e-10,1.644256e-10,1.632658e-10,1.636569e-10,1.650793e-10


In [9]:
segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (10*TR_C*W/3) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 14.8 *1e-12 #measured
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.23367685e-09, 2.81562843e-09, 1.65735352e-09, 1.19404273e-09,
       1.19097594e-09, 1.62579040e-09, 2.67477234e-09, 4.79601128e-09])

In [10]:
delay_total = delay_decoder.copy()
delay_total = delay_total + delay_WL
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.417850e-09,2.967624e-09,1.787214e-09,1.309409e-09,1.298057e-09,1.729804e-09,2.779797e-09,4.904917e-09
NAND3-NOR2 Stage2,5.402066e-09,2.950453e-09,1.769090e-09,1.290661e-09,1.278953e-09,1.710567e-09,2.760603e-09,4.885890e-09
NAND6-INV Stage4,5.389931e-09,2.961992e-09,1.795685e-09,1.326238e-09,1.319190e-09,1.752412e-09,2.801927e-09,4.925139e-09
NAND3-NOR2 Stage4,5.370674e-09,2.942525e-09,1.776046e-09,1.306469e-09,1.299336e-09,1.732524e-09,2.782050e-09,4.905305e-09
NAND6-INV Stage6,5.410493e-09,2.985672e-09,1.821584e-09,1.353599e-09,1.347369e-09,1.780885e-09,2.830304e-09,4.953141e-09
NAND3-NOR2 Stage6,5.389883e-09,2.965062e-09,1.800974e-09,1.332989e-09,1.326759e-09,1.760275e-09,2.809694e-09,4.932531e-09
NAND6-INV Stage8,5.435263e-09,3.011727e-09,1.848609e-09,1.381303e-09,1.375476e-09,1.809143e-09,2.858512e-09,4.981158e-09
NAND3-NOR2 Stage8,5.415370e-09,2.991775e-09,1.828606e-09,1.361258e-09,1.355402e-09,1.789056e-09,2.838429e-09,4.961091e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.403941e-09,2.972104e-09,1.803476e-09,1.332741e-09,1.325068e-09,1.758083e-09,2.807665e-09,4.931146e-09
std,2.018557e-11,2.285547e-11,2.741902e-11,3.083205e-11,3.276360e-11,3.345389e-11,3.322855e-11,3.234522e-11
min,5.370674e-09,2.942525e-09,1.769090e-09,1.290661e-09,1.278953e-09,1.710567e-09,2.760603e-09,4.885890e-09
25%,5.389919e-09,2.959107e-09,1.784422e-09,1.308674e-09,1.299016e-09,1.731844e-09,2.781487e-09,4.905208e-09
50%,5.406279e-09,2.966343e-09,1.798330e-09,1.329613e-09,1.322975e-09,1.756344e-09,2.805811e-09,4.928835e-09
75%,5.415990e-09,2.987198e-09,1.823340e-09,1.355514e-09,1.349377e-09,1.782928e-09,2.832335e-09,4.955128e-09
max,5.435263e-09,3.011727e-09,1.848609e-09,1.381303e-09,1.375476e-09,1.809143e-09,2.858512e-09,4.981158e-09


In [12]:
H

array([1.93711637, 0.98420621, 0.51423275, 0.28841241, 0.18846549,
       0.15682483, 0.16693098, 0.20864965])

In [13]:
f_2

,1,2,4,8,16,32,64,128
NAND6-INV,12.448667,8.873359,6.413939,4.803436,3.882942,3.542031,3.654378,4.085581
NAND3-NOR2,12.984861,9.255556,6.690203,5.010331,4.050190,3.694595,3.811781,4.261557


In [14]:
W

array([0.60249632, 0.85205847, 1.20499264, 1.70411693, 2.40998528,
       3.40823387, 4.81997056, 6.81646773])

In [15]:
2.40998528 * 7/3

5.623298986666666